In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [2]:
## TODO make it DRY
# img_src = './data/img_data_seg_1/'
# img_dst = './data/img_preprocessed_set_1/'
img_src = './data/img_data_seg_2/'
img_dst = './data/img_preprocessed_set_2/'

In [3]:
file_name_list = os.listdir(img_src)
print(f"Number of images in dir {len(file_name_list)}")

Number of images in dir 2751


In [4]:
## Get estimates:
## number of images per class
## number of images per sequence per class

In [5]:
class_bins = {0: 'shake_hands',
             1: 'hug',
             2: 'kick',
             3: 'point',
             4: 'punch',
             5: 'push'}

In [6]:
# a dictionary to hold all filenames by respective classes
all_files = [f'files_of_{cls}' for cls in list(class_bins.keys())]
all_files = dict.fromkeys(all_files)
print(all_files)

{'files_of_0': None, 'files_of_1': None, 'files_of_2': None, 'files_of_3': None, 'files_of_4': None, 'files_of_5': None}


In [7]:
def getClassNum(video_file_name):
    """does some string operations to obtain class num"""
    file_parts = video_file_name.split('_')
    class_of_file = int(file_parts[0])
    return class_of_file

In [8]:
def closureFilterClass(class_num):
    def filterByClass(file_name):
        if getClassNum(file_name)==class_num:
            return True
        else:
            return False
    return filterByClass

In [9]:
for key, val in class_bins.items():
    filter_c = closureFilterClass(key)
    temp = list(filter(filter_c, file_name_list))
    dict_key_name = f"files_of_{key}"
    all_files[dict_key_name] = temp

In [10]:
print("number of images per class : ")
for key, val in class_bins.items():
    dict_key_name = f"files_of_{key}"
    print(f"{val}, ({key}) \t--> {len(all_files[dict_key_name])}")

number of images per class : 
shake_hands, (0) 	--> 644
hug, (1) 	--> 697
kick, (2) 	--> 448
point, (3) 	--> 0
punch, (4) 	--> 436
push, (5) 	--> 526


In [11]:
def readImg(file_name, loc_src=img_src):
    file_loc = loc_src + file_name
    img = cv2.imread(file_loc)
    return img

In [12]:
def imgPreprocessor(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Size of original images
    # max-min range --> (332-216, 612, 244)
    # avg range --> (260, 380)
    img_gray = cv2.resize(img_gray, (70, 95))
    img_gauss = cv2.GaussianBlur(img_gray, (5,5), 0)
    img_norm = np.empty_like((img_gauss))
    img_norm = cv2.normalize(img_gauss, img_norm, 0, 255, cv2.NORM_MINMAX)
    return img_norm

In [13]:
def writeImg(img, file_name, loc_dst=img_dst):
    file_loc = loc_dst + file_name
    cv2.imwrite(file_loc, img)

In [14]:
def preprocessAndWrite(file_name):
    img = readImg(file_name)
    img_p = imgPreprocessor(img)
    writeImg(img_p, file_name)

In [15]:
# write all images into new directory after preprocessing
for img_file in file_name_list:
    preprocessAndWrite(img_file)